In [30]:
import numpy as np
import pandas as pd

from metrics import Metric, calc_metrics, isna
%load_ext autoreload
%autoreload 2

In [3]:
table = pd.read_excel('../data.xlsx')
table.head()

,Название,Специальность,Телефон,email,Сайт,vk.com,instagram.com,facebook.com,Место,Метро,Расписание,Прейскурант
0,"Студия художественного творчества ""ПАЛИТРА""",Рисование,79175367002,izozavart@yandex.ru,www.galleryart-mos.com,https://vk.com/vserisujut,https://www.instagram.com/izozavart/,None,"Литовский бульвар, 19",Ясенево,По телефону,По телефону
1,Школа рисования для взрослых Матита,Рисование,74993978111,art-matita@mail.ru,http://art-matita.ru/blog/,https://vk.com/artmatita,http://instagram.com/artmatita,None,"Цветной бульвар д. 16/1, подъезд 2, Офис 12 , ...","Цветной бульвар, Трубная",По телефону,По телефону
2,Художественная школа «Дом ИЗО»,Рисование,79152554378,nitaso@mail.ru,http://domizo.ru/shkola-zhivopisi-dlya-vzroslih,None,https://www.instagram.com/domizo.ru/,None,"ул.Тверская дом 6,строение 6",Охотный ряд,http://domizo.ru/schedule,http://domizo.ru/cost
3,Школа Искусств Перотти на Динамо,Рисование,74959201280,info@perottischool.ru,https://www.perottischool.ru/,https://vk.com/perottiartschool,https://www.instagram.com/perottiartschool/,https://www.facebook.com/schoolperotti/,"ул. Расковой, дом 16/18","Белорусская, Динамо",https://www.perottischool.ru/kalendar-perotti/,https://www.perottischool.ru/stoimost/
4,Школа Искусств Перотти на Юго-Западе,Рисование,74959201278,info@perottischool.ru,https://www.perottischool.ru/,https://vk.com/perottiartschool,https://www.instagram.com/perottiartschool/,https://www.facebook.com/schoolperotti/,"ул. Никулинская дом 6, корп. 3",Юго-Западная,https://www.perottischool.ru/kalendar-perotti/,https://www.perottischool.ru/stoimost/


In [4]:
# Строчки из None
table.drop(range(244, 263), inplace=True)

In [5]:
result = pd.read_csv('../result.csv')
result.head()

,Unnamed: 0,url,email,phone,vk.com,instagram.com,facebook.com,image_source,schedule,title,ok.ru,address,metro_station
0,0,http://www.galleryart-mos.com,izozavart@gmail.com,+7 917 536 70 02,NaN,https://www.instagram.com/izozavart/,https://www.facebook.com/palitrayasenevo/,NaN,NaN,Купить картину в Москве | Галерея художников М...,NaN,NaN,NaN
1,0,http://art-matita.ru/blog/,art-matita@mail.ru,+7 (925) 488-82-47,NaN,NaN,NaN,https://art-matita.ru/blog/wp-content/uploads/...,NaN,Школа рисования и живописи в Москве: студия Ar...,NaN,NaN,NaN
2,0,http://domizo.ru/shkola-zhivopisi-dlya-vzroslih,nitaso@mail.ru,+7 (915) 255-43-78,NaN,https://www.instagram.com/domizo.ru/,NaN,https://domizo.ru/wp-content/uploads/2018/05/i...,https://domizo.ru/schedule,Обучение рисованию и живописи для взрослых,NaN,NaN,NaN
3,0,https://www.perottischool.ru/,info@perottischool.ru,8(495)920-12-80,https://vk.com/perottiartschool,https://www.instagram.com/perottiartschool/,https://www.facebook.com/schoolperotti/,https://www.perottischool.ru/wp-content/upload...,NaN,— курсы рисования и живописи в Москве,NaN,Расковой ул. 16,NaN
4,0,https://izokurs.ru/,123@izokurs.ru,+7(499) 551-56-15,https://vk.com/izokurs,https://www.instagram.com/izokurs/,https://www.facebook.com/izokurs/,https://izokurs.ru/assets/tpl/images/maintop/s...,NaN,Школа Рисования для Взрослых | Рисунок и Живоп...,NaN,"Большая Татарская улица, 35, стр. 7-9",Новокузнецкая


In [10]:
# Техническая колонка
result.drop(columns='Unnamed: 0', inplace=True)

Нужно, чтобы соответствующие колонки таблиц назывались одинаково

In [47]:
table.rename(columns={
    'Сайт' : 'url',
    'Метро' : 'metro_station',
    'Место' : 'address',
    'Расписание' : 'schedule',
    'Телефон' : 'phone'
}, inplace=True)

In [48]:
table.head()

,Название,Специальность,phone,email,url,vk.com,instagram.com,facebook.com,address,metro_station,schedule,Прейскурант
0,"Студия художественного творчества ""ПАЛИТРА""",Рисование,79175367002,izozavart@yandex.ru,www.galleryart-mos.com,https://vk.com/vserisujut,https://www.instagram.com/izozavart/,None,"Литовский бульвар, 19",Ясенево,По телефону,По телефону
1,Школа рисования для взрослых Матита,Рисование,74993978111,art-matita@mail.ru,http://art-matita.ru/blog/,https://vk.com/artmatita,http://instagram.com/artmatita,None,"Цветной бульвар д. 16/1, подъезд 2, Офис 12 , ...","Цветной бульвар, Трубная",По телефону,По телефону
2,Художественная школа «Дом ИЗО»,Рисование,79152554378,nitaso@mail.ru,http://domizo.ru/shkola-zhivopisi-dlya-vzroslih,None,https://www.instagram.com/domizo.ru/,None,"ул.Тверская дом 6,строение 6",Охотный ряд,http://domizo.ru/schedule,http://domizo.ru/cost
3,Школа Искусств Перотти на Динамо,Рисование,74959201280,info@perottischool.ru,https://www.perottischool.ru/,https://vk.com/perottiartschool,https://www.instagram.com/perottiartschool/,https://www.facebook.com/schoolperotti/,"ул. Расковой, дом 16/18","Белорусская, Динамо",https://www.perottischool.ru/kalendar-perotti/,https://www.perottischool.ru/stoimost/
4,Школа Искусств Перотти на Юго-Западе,Рисование,74959201278,info@perottischool.ru,https://www.perottischool.ru/,https://vk.com/perottiartschool,https://www.instagram.com/perottiartschool/,https://www.facebook.com/schoolperotti/,"ул. Никулинская дом 6, корп. 3",Юго-Западная,https://www.perottischool.ru/kalendar-perotti/,https://www.perottischool.ru/stoimost/


Сами метрики:

In [130]:
res, history = calc_metrics(
    data_pred=result,
    data_true=table,
    key='url',
    columns=['email', 'phone', 'vk.com', 'instagram.com', 'facebook.com',
       'schedule', 'address', 'metro_station'],
    preprocs={
        'url': 'url',
        'phone': 'phone',
        'vk.com': 'url',
        'instagram.com': 'url',
        'facebook.com': 'url',
        'schedule': 'url'
    },
    metrics=['precision', 'recall'],
    save_history=True
)
res

,email,phone,vk.com,instagram.com,facebook.com,schedule,address,metro_station
precision,0.812500,0.388889,0.928571,0.8125,0.846154,0.571429,0.0,0.333333
recall,0.619048,0.280000,0.650000,0.6500,0.611111,0.153846,0.0,0.076923


Необходимо посмотреть на ошибки чтобы понимать, это действительно ошибки парсера или синонимы.

In [131]:
cols = ['email', 'phone', 'vk.com', 'instagram.com', 'facebook.com',
       'schedule', 'address', 'metro_station']
for col in cols:
    print(f'---------------------------{col}----------------------------------')
    for z in history[(col, 'precision')]:
        if z[0] == 'Wrong':
            print('Pred:', repr(z[1]))
            print('True:', z[2])
            print('')

---------------------------email----------------------------------
Pred: 'izozavart@gmail.com'
True: {'izozavart@yandex.ru'}

Pred: 'contract@tancy.pro'
True: {'info@tancy.pro'}

Pred: 'hr@chesstogo.ru'
True: {'mail@chesstogo.ru'}

---------------------------phone----------------------------------
Pred: '89254888247'
True: {'84993978111'}

Pred: '84951201714'
True: {'74993808370доб.1', '74993808370доб.3', '74993808370доб.2'}

Pred: '89776514410'
True: {'74951424428;79776514410'}

Pred: '83422364030'
True: {'84954778927'}

Pred: '84952043040'
True: {'89168094259'}

Pred: '89165765775'
True: {'79167806565;79165765775'}

Pred: '84994080323'
True: {'89055743106'}

Pred: '89160236434'
True: {'79771095510;79160236434'}

Pred: '84951512152'
True: {'74951512152;79267000101'}

Pred: '89166398005'
True: {'79166398005;79253782124'}

Pred: '89162446713'
True: {'79162446713;79037672274'}

---------------------------vk.com----------------------------------
Pred: 'vk.com/music_school_db'
True: {'vk.c

Видим серьезные проблемы с синонимами, которые сложно правильно предобработать, в колонках phone, facebook, schedule, address. Этим результатам нельзя доверять.

Стоит также отметить, что ошибка в ВК-страничке - это ошибка в исходной таблице (вернее, в ней устаревшая ссылка), а ошибки в инстаграме - это неудачно выбранные парсером странички.

В остальных ситуациях мы видим очень высокую точность и полноту порядка 60 %. То есть если парсер что-то нашел - он скорее всего прав, но он часто пропускает находимую информацию. Исключение - станция метро, ее он ищет довольно таки плохо.